In [15]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

SRC = (Path.cwd().parent / "src").resolve()
sys.path.insert(0, str(SRC))

import importlib
import functions
importlib.reload(functions)   

from functions import *       


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
# ================================================
# 1. session selection (main or external dataset):
# ================================================
session = "m2" # AUF(main), ZU(TUM.rep), m1 (Vienna.rep session 1) or m2 (Vienna.rep session 2)
DIR = f'/RAID1/jupytertmp/mi/{session}/'
nrois = 360


# ================================================
# 2. select if you want to include limbic or not:
# ================================================

# including limbic:
limb_ind = []
LIMB = 'with'

# excluding limbic:
# limb_ind = np.array([88, 90, 92, 93, 110, 118, 120, 122, 131, 135, 165, 166, 172, 268, 270, 272, 273, 290, 298, 300, 302, 311, 314, 315, 346, 352])-1
# LIMB = 'without'





sub = os.listdir(DIR)
sub = sorted([int(s.split('-')[1]) for s in sub if s.startswith('sub-') and s.split('-')[1].isdigit()])

if session == "ZU":
    
    sub.remove(14)


if session == 'AUF':
    sub = [3,7,12,14,17,20,23,25,26,28,29,30,31,32,33,35,36,37,38]



net_label = pd.read_csv(f'../data/external/mmp2yeo7nw_mapping.csv')
net_label = net_label.drop(limb_ind,axis=0)
net_names = pd.unique(net_label['yeo_7_nw']) 
net_label.reset_index(drop=True, inplace=True)
network_to_number = {network: i + 1 for i, network in enumerate(net_label['yeo_7_nw'].unique())}
net_label['network_number'] = net_label['yeo_7_nw'].map(network_to_number)

# Functional and structural connectivity

In [38]:
file_path = f'../data/processed/matrices_data_{session}_{LIMB}_limbic.pkl'
data_input = Functional_conn(DIR, session, sub, limb_ind, nrois)   
    
with open(file_path, 'wb') as file:
    pickle.dump(data_input, file, protocol=pickle.HIGHEST_PROTOCOL)




    

# Mwc and DC calcualtions

In [39]:

## with structural connectivity masking:
# data_avg_path = f'../data/processed/DATA_avg_wSC_{session}_{LIMB}_limb.csv'
# data_sub_path = f'../data/processed/data_single_sub_wSC_{session}_{LIMB}_limb.pkl'
# data_avg_pr_path = f'../data/processed/DATA_avg_wSC_pr_{session}_{LIMB}_limb.csv'
# data_sub_pr_path = f'../data/processed/data_single_sub_wSC_pr_{session}_{LIMB}_limb.pkl'

## without structural connectivity masking:
data_avg_pr_path = f'../data/processed/DATA_avg_woSC_pr_{session}_{LIMB}_limb.csv'
data_sub_pr_path = f'../data/processed/data_single_sub_woSC_pr_{session}_{LIMB}_limb.pkl'
data_avg_path = f'../data/processed/DATA_avg_woSC_{session}_{LIMB}_limb.csv'
data_sub_path = f'../data/processed/data_single_sub_woSC_{session}_{LIMB}_limb.pkl'


num_voxels = data_input['num_voxels']
MIconn_allsub = data_input['MIconn_allsub']
SCconn_allsub = data_input['SCconn_allsub']
Pearconn_allsub = data_input['Pearconn_allsub']
edata_medianallsub = data_input['edata_medianallsub']




#--------------------------------------------------------------------------------------------
#-------------- MWC calculation  ----------------
#--------------------------------------------------------------------------------------------
scmask = 1
nrois_rem = nrois - len(limb_ind) 
thr_sc = 0.1
sub_size = MIconn_allsub.shape[2]

if session == "AUF":
 
    scmask = 1
else:
    scmask = 0


MIconn_allsub[MIconn_allsub < 0.01] = 0
Conn_Mat = MIconn_allsub
thr_FC = 1



[data_avg, data_single_sub] = IC_calculation(Conn_Mat, edata_medianallsub, SCconn_allsub, thr_sc, thr_FC, sub_size, scmask, nrois_rem,net_label)
DATA_avg = pd.concat([data_avg , net_label], axis =1)


DATA_avg.to_csv(data_avg_path, index=False)
with open(data_sub_path, 'wb') as file:
    pickle.dump(data_single_sub, file, protocol=pickle.HIGHEST_PROTOCOL)



#--------------------------------------------------------------------------------------------
#-------------- DC calculation ----------------
#--------------------------------------------------------------------------------------------

scmask = 1
thr_sc = 0.1
sub_size = MIconn_allsub.shape[2]
nrois_rem = nrois - len(limb_ind) 



thr_FC = 0.2 
Conn_Mat = Pearconn_allsub
Conn_Mat[Conn_Mat < 0] = 0
#Conn_Mat = np.abs(Conn_Mat)

#DC with MI:
# thr_FC = 1
# Conn_Mat = MIconn_allsub
# Conn_Mat[Conn_Mat < 0.001] = 0


[data_avg_pr, data_single_sub_pr] = IC_calculation(Conn_Mat, edata_medianallsub, SCconn_allsub, thr_sc, thr_FC, sub_size, scmask, nrois_rem,net_label)
DATA_avg_pr = pd.concat([data_avg_pr , net_label], axis =1)

DATA_avg_pr.to_csv(data_avg_pr_path, index=False)
with open(data_sub_pr_path, 'wb') as file:
    pickle.dump(data_single_sub_pr, file, protocol=pickle.HIGHEST_PROTOCOL)


